In [8]:
# ## 디렉토리 설정
import os
os.getcwd()

os.chdir("../GitCode")
os.getcwd()

'D:\\user\\Desktop\\Project_1\\GitCode'

In [1]:
## 패키지 임포팅
import pandas as pd
import numpy as np

In [20]:
## 데이터로딩
dir="../prosumer_behaviour/"
train=pd.read_csv(dir+"train.csv")
client=pd.read_csv(dir+"client.csv")
gas=pd.read_csv(dir+"gas_prices.csv")
ele=pd.read_csv(dir+"electricity_prices.csv")
forecast=pd.read_csv(dir+"forecast_weather.csv")
historical=pd.read_csv(dir+"historical_weather.csv")
map=pd.read_csv(dir+"weather_station_to_county_mapping.csv")
print(train.shape)

(2018352, 9)


In [13]:
print(train.columns)
print(client.columns)
print(gas.columns)
print(ele.columns)
print(forecast.columns)
print(historical.columns)

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')
Index(['product_type', 'county', 'eic_count', 'installed_capacity',
       'is_business', 'date', 'data_block_id'],
      dtype='object')
Index(['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'origin_date', 'data_block_id'],
      dtype='object')
Index(['forecast_date', 'euros_per_mwh', 'origin_date', 'data_block_id'], dtype='object')
Index(['latitude', 'longitude', 'origin_datetime', 'hours_ahead',
       'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component', 'data_block_id', 'forecast_datetime',
       'direct_solar_radiation', 'surface_solar_radiation_downwards',
       'snowfall', 'total_precipitation'],
      dtype='object')
Index(['datetime', 'temperature', 'dewpoint', 'rai

In [21]:
# train 데이터의 날짜 자료형: object->datetime
train['datetime'] = pd.to_datetime(train['datetime'])

# datetime의 특성을 컬럼으로 추가
train['dt_date'] = train['datetime'].dt.normalize() # 2021-09-01 11:00:00 -> 2021-09-01 00:00:00
train['dt_year'] = train['datetime'].dt.year
train['dt_quarter'] = train['datetime'].dt.quarter
train['dt_month'] = train['datetime'].dt.month
train['dt_week'] = train['datetime'].dt.isocalendar().week # iso 달력을 기준으로 week 정보 추가
train['dt_day_of_month']  = train['datetime'].dt.day
train['dt_hour'] = train['datetime'].dt.hour

# 요일 특성 담아주기
train['day_of_year'] = train['datetime'].dt.day_of_year
train['day_of_week'] = train['datetime'].dt.day_of_week

In [23]:
## train data preprocessing
print(train.isna().sum())
print(train[train['target'].isna()==True]['datetime'].value_counts())
# target NaN for SummerTime -> fillna with the month's at 3 o'clock by prediction unit id 
for c in [0,1]:
    mean_train=train[train['is_consumption']==c].groupby(['dt_month','dt_hour','prediction_unit_id'])[['target']].mean().reset_index()
    for m in [3,10]:
        pui =train[((train['dt_month']==m)&(train['dt_hour']==3)&(train['target'].isna()==True)&(train['is_consumption']==c))]['prediction_unit_id'].values
        for p in pui: 
            indice=train[((train['is_consumption']==c)&(train['prediction_unit_id']==p)&(train['dt_month']==m)&(train['dt_hour']==3)&(train['target'].isna()))].index
            for idx in indice:
                train.loc[idx,'target']=mean_train[((mean_train['prediction_unit_id']==p)&(mean_train['dt_month']==m)&(mean_train['dt_hour']==3))]['target'].values[0]

county                  0
is_business             0
product_type            0
target                528
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dt_date                 0
dt_year                 0
dt_quarter              0
dt_month                0
dt_week                 0
dt_day_of_month         0
dt_hour                 0
day_of_year             0
day_of_week             0
dtype: int64
datetime
2022-10-30 03:00:00    136
2022-03-27 03:00:00    134
2023-03-26 03:00:00    132
2021-10-31 03:00:00    126
Name: count, dtype: int64


In [24]:
print(train.isna().sum())

county                0
is_business           0
product_type          0
target                0
is_consumption        0
datetime              0
data_block_id         0
row_id                0
prediction_unit_id    0
dt_date               0
dt_year               0
dt_quarter            0
dt_month              0
dt_week               0
dt_day_of_month       0
dt_hour               0
day_of_year           0
day_of_week           0
dtype: int64


In [28]:
## merging train and client

df_merging1=train.merge(client, how='left', on=['county','is_business','product_type','data_block_id'])
display(df_merging1.isna().sum())
df_merging2= df_merging1.drop('date', axis=1) # client 데이터는 트레인 데이터에 비하여 2일 늦음. 즉, 그제의 클라이언트 자료에 기반하여 현재의 target 값이 결정되는 구조

# dbi 0과 1의 클라이언트 결측치를 채우기 위해서는 각각의 prediction_unit_id에 알맞은 값을 넣어줘야함(dbi=2에 해당하는 eic와 capacity 값 넣어줌)
for p in np.unique(df_merging2['prediction_unit_id']):
    for d in [1,0]:
        for col in ['eic_count','installed_capacity']:
            nan_idx=df_merging2[((df_merging2[col].isna()==True)&(df_merging2['data_block_id']==d)&(df_merging2['prediction_unit_id']==p))].index.values.tolist()
            df_merging2.loc[nan_idx,col]=df_merging2[((df_merging2['data_block_id']==d+1)&(df_merging2['prediction_unit_id']==p))][col].values # bfill method 적용하여 채우기


display(df_merging2.isna().sum())
# 여전히 남아있는 2784개의 NaN 값에 해당하는 prediction_unit_id와 data_block_id 확인
nan_type=np.unique(df_merging2[df_merging2['eic_count'].isna()==True]['prediction_unit_id'])
print(nan_type)
nan_dbi=np.unique(df_merging2[df_merging2['eic_count'].isna()]['data_block_id'])[::-1] # eic가 NaN인 값의 data block id를 역순으로 가져옴. 이걸 기준으로 eic 나 capacity NAN인 데이터의 dbi 가져올 수 있음
print(nan_dbi)

for p in nan_type:
    nan_dbi_p=(np.unique(df_merging2[((df_merging2['prediction_unit_id']==p) & (df_merging2['eic_count'].isna()==True))]['data_block_id']))[::-1]
    print(p, nan_dbi_p)
    for d in nan_dbi_p:
        for col in ['eic_count','installed_capacity']:
            try:
                nan_idx=df_merging2[((df_merging2[col].isna()==True)&(df_merging2['data_block_id']==d)&(df_merging2['prediction_unit_id']==p))].index.values.tolist()
                df_merging2.loc[nan_idx,col]=df_merging2[((df_merging2['data_block_id']==d+1)&(df_merging2['prediction_unit_id']==p))][col].values # bfill method
            except:
                print(f'bfill not available type: {p}, dbi: {d}')
                continue

display(df_merging2.isna().sum())

# 남은 144개의 결측치는 대체 불가함 ->drop
df_merging3=df_merging2.dropna()
print(df_merging3.isna().sum())

county                   0
is_business              0
product_type             0
target                   0
is_consumption           0
datetime                 0
data_block_id            0
row_id                   0
prediction_unit_id       0
dt_date                  0
dt_year                  0
dt_quarter               0
dt_month                 0
dt_week                  0
dt_day_of_month          0
dt_hour                  0
day_of_year              0
day_of_week              0
eic_count             8640
installed_capacity    8640
date                  8640
dtype: int64

county                   0
is_business              0
product_type             0
target                   0
is_consumption           0
datetime                 0
data_block_id            0
row_id                   0
prediction_unit_id       0
dt_date                  0
dt_year                  0
dt_quarter               0
dt_month                 0
dt_week                  0
dt_day_of_month          0
dt_hour                  0
day_of_year              0
day_of_week              0
eic_count             2784
installed_capacity    2784
dtype: int64

[21 26 41 44 47 61 62 63 64 65 66 67 68]
[612 611 608 607 606 603 602 596 595 591 590 584 583 580 573 572 541 540
 533 532 519 518 514 513 458 427 426 396 395 366 365 274 273 216 215 204
 203 173 172 154 153 123 122  92  91  31  30]
21 [154 153]
26 [519 518 366 365 274 273 216 215 173 172]
41 [612 611 541 540 458 396 395 204 203]
44 [607 606 591 590 123 122]
47 [603 602 596 595 584 583 580 514 513]
bfill not available type: 47, dbi: 596
bfill not available type: 47, dbi: 596
bfill not available type: 47, dbi: 580
bfill not available type: 47, dbi: 580
61 [31 30]
62 [31 30]
63 [31 30]
64 [92 91]
65 [123 122]
66 [123 122]
67 [154 153]
68 [608 607 573 572 533 532 427 426]


county                  0
is_business             0
product_type            0
target                  0
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dt_date                 0
dt_year                 0
dt_quarter              0
dt_month                0
dt_week                 0
dt_day_of_month         0
dt_hour                 0
day_of_year             0
day_of_week             0
eic_count             144
installed_capacity    144
dtype: int64

county                0
is_business           0
product_type          0
target                0
is_consumption        0
datetime              0
data_block_id         0
row_id                0
prediction_unit_id    0
dt_date               0
dt_year               0
dt_quarter            0
dt_month              0
dt_week               0
dt_day_of_month       0
dt_hour               0
day_of_year           0
day_of_week           0
eic_count             0
installed_capacity    0
dtype: int64


In [30]:
## merge gas and ele
# df gas avg price
gas['euros_per_mwh']=gas['lowest_price_per_mwh']/2 + gas['highest_price_per_mwh']/2

# merge
df_prices1=pd.merge(ele,gas, how='inner', on='data_block_id', suffixes=('_electricity','_gas'))
# display(df_prices1)

In [31]:
## merge train + prices
# train 과의 결합을 위하여 datetime column 추가
df_prices1['datetime']=pd.to_datetime(df_prices1['forecast_date_electricity'])+pd.DateOffset(hours=24)
df_prices2=df_prices1[['datetime','data_block_id','euros_per_mwh_electricity','euros_per_mwh_gas']]

# merge
df_merging4=df_merging3.merge(df_prices2, how='left', on=['datetime','data_block_id'])
# display(df_merging4)

In [33]:
print(df_merging4.isna().sum())
print(df_merging4[df_merging4['euros_per_mwh_gas'].isna()==True][['datetime','data_block_id']].values)

# gas 결측치 처리: data block id == 0 이었던 부분 및 서머타임으로 인해 발생한 결측치 -> bfill 처리
df_merging4['euros_per_mwh_gas']=df_merging4['euros_per_mwh_gas'].fillna(method='bfill')
print(df_merging4.isna().sum())

# electricity의 경우 시간을 고려하여 결측치를 처리해주어야함 -> 다음날 같은 시간의 가격으로 대체
# df_merging4[df_merging4['euros_per_mwh_electricity']==True]

nan_ele_idx=df_merging4[df_merging4['euros_per_mwh_electricity'].isna()==True].index.values.tolist()
for i in nan_ele_idx:
    # print(i)
    d=df_merging4['data_block_id'].loc[i]
    h=df_merging4['dt_hour'].loc[i]
    df_merging4.loc[i,'euros_per_mwh_electricity']=df_merging4[((df_merging4['data_block_id']==d+1)&(df_merging4['dt_hour']==h))]['euros_per_mwh_electricity'].values[0]
    # print(df_merging4[((df_merging4['data_block_id']==d+1)&(df_merging4['dt_hour']==h))]['euros_per_mwh_electricity'].values[0])
display(df_merging4.isna().sum())

county                          0
is_business                     0
product_type                    0
target                          0
is_consumption                  0
datetime                        0
data_block_id                   0
row_id                          0
prediction_unit_id              0
dt_date                         0
dt_year                         0
dt_quarter                      0
dt_month                        0
dt_week                         0
dt_day_of_month                 0
dt_hour                         0
day_of_year                     0
day_of_week                     0
eic_count                       0
installed_capacity              0
euros_per_mwh_electricity    3196
euros_per_mwh_gas            3196
dtype: int64
[[Timestamp('2021-09-01 00:00:00') 0]
 [Timestamp('2021-09-01 00:00:00') 0]
 [Timestamp('2021-09-01 00:00:00') 0]
 ...
 [Timestamp('2023-03-27 02:00:00') 572]
 [Timestamp('2023-03-27 02:00:00') 572]
 [Timestamp('2023-03-27 02:00:00') 572]]

county                       0
is_business                  0
product_type                 0
target                       0
is_consumption               0
datetime                     0
data_block_id                0
row_id                       0
prediction_unit_id           0
dt_date                      0
dt_year                      0
dt_quarter                   0
dt_month                     0
dt_week                      0
dt_day_of_month              0
dt_hour                      0
day_of_year                  0
day_of_week                  0
eic_count                    0
installed_capacity           0
euros_per_mwh_electricity    0
euros_per_mwh_gas            0
dtype: int64

In [35]:
## mapping county to weather information
# 카운티 위도 경도를 날씨정보와 동일하게 소수점 1자리 까지만 표기
map['latitude'], map['longitude']= map['latitude'].apply(lambda x: np.round(x,1)),map['longitude'].apply(lambda x: np.round(x,1))
df_fweather_merging1=forecast.merge(map, on=['latitude','longitude']).dropna()
df_hweather_merging1=historical.merge(map, on=['latitude','longitude']).dropna()

In [36]:
## unify time columns into 'datetime'
df_fweather_merging1['origin_datetime']=pd.to_datetime(df_fweather_merging1['origin_datetime'])
df_fweather_merging1['forecast_datetime']=pd.to_datetime(df_fweather_merging1['forecast_datetime'])

df_hweather_merging1['datetime']=pd.to_datetime(df_hweather_merging1['datetime'])
df_hweather_merging1=df_hweather_merging1.rename(columns={'datetime':'historical_datetime'})
df_hweather_merging1['forecast_datetime']=df_hweather_merging1['historical_datetime']+pd.DateOffset(hours=48) # in order to match data
# in one record, the datetime means when the target is produced or consumed. at the same datetime, forecast weather information is prediction value, and all other data are 48 hours before.


In [37]:
## filter weather forecast data. since we want to know T+48 data.
df_fweather_merging1[((df_fweather_merging1['hours_ahead']>=22)&(df_fweather_merging1['hours_ahead']<=46)&(df_fweather_merging1['county']==0))]
df_fweather_merging2=df_fweather_merging1[((df_fweather_merging1['hours_ahead']>21)&(df_fweather_merging1['hours_ahead']<46))]

In [38]:
## get the average of several forecast coordinates in one county into one point. 
df_fweather_merging3=df_fweather_merging2.groupby(['forecast_datetime','county_name']).mean().reset_index().drop(['latitude','longitude'],axis=1)
df_hweather_merging2=df_hweather_merging1.groupby(['forecast_datetime','county_name']).mean().reset_index().drop(['latitude','longitude'],axis=1)

In [39]:
## column equalization between forecast and historical since the two dataframe uses different kind of weather data
# print(df_fweather_merging3.columns)
df_fweather_merging4=df_fweather_merging3.drop(['county_name','hours_ahead','origin_datetime','10_metre_u_wind_component','10_metre_v_wind_component'],axis=1)
rain=df_fweather_merging4['total_precipitation']-df_fweather_merging4['snowfall']
df_fweather_merging4['total_precipitation']=rain
df_fweather_merging4.rename(columns={'total_precipitation':'rain'}, inplace=True)
# print(df_fweather_merging4.columns)

# for prediction of solar energy production wind is least related.
# print(df_hweather_merging2.columns)
df_hweather_merging3=df_hweather_merging2.drop(['county_name', 'surface_pressure', 'winddirection_10m','windspeed_10m','data_block_id'],axis=1)
# print(df_hweather_merging3.columns)

In [41]:
## merge forecast data with historical data
df_weather=df_fweather_merging4.merge(df_hweather_merging3, how='left', on=['forecast_datetime','county'], suffixes=('_f','_h'))

## merge train + weather
df_merging5=pd.merge(df_merging4,df_weather, how='left', left_on=['datetime','county'],
                     right_on=['forecast_datetime','county'],suffixes=('','_weather'))

## the count 12 which is 'unknown county' means overseas, so should be removed.
# print(df_merging5.drop(df_merging5[df_merging5['county']==12].index).isna().sum())
df_merging6=df_merging5.drop(df_merging5[df_merging5['county']==12].index)
df_merging6

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,dt_date,...,dewpoint_h,rain_h,snowfall_h,cloudcover_total_h,cloudcover_low_h,cloudcover_mid_h,cloudcover_high_h,shortwave_radiation,direct_solar_radiation_h,diffuse_radiation
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018203,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31,...,6.98,0.0,0.0,21.2,2.8,16.4,29.8,0.0,0.0,0.0
2018204,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31,...,6.98,0.0,0.0,21.2,2.8,16.4,29.8,0.0,0.0,0.0
2018205,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31,...,6.98,0.0,0.0,21.2,2.8,16.4,29.8,0.0,0.0,0.0
2018206,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31,...,6.98,0.0,0.0,21.2,2.8,16.4,29.8,0.0,0.0,0.0


In [55]:
print(df_merging6.isna().sum())
print(np.unique(df_merging6[df_merging6['forecast_datetime'].isna()==True]['datetime']))
# NaNs in weather data are because of summer time and no info of the first and second day. so will be removed.
df_merging7=df_merging6.drop(df_merging6[df_merging6['data_block_id']==0].index).drop(df_merging6[df_merging6['data_block_id']==1].index)
df_merging8=df_merging7.dropna()
print(df_merging8.isna().sum())

county                                  0
is_business                             0
product_type                            0
target                                  0
is_consumption                          0
datetime                                0
data_block_id                           0
row_id                                  0
prediction_unit_id                      0
dt_date                                 0
dt_year                                 0
dt_quarter                              0
dt_month                                0
dt_week                                 0
dt_day_of_month                         0
dt_hour                                 0
day_of_year                             0
day_of_week                             0
eic_count                               0
installed_capacity                      0
euros_per_mwh_electricity               0
euros_per_mwh_gas                       0
forecast_datetime                    3142
temperature_f                     

In [58]:
## column manipulation
df_merging9=df_merging8.drop(['data_block_id_weather','historical_datetime','forecast_datetime'], axis=1).dropna().rename(columns={'shortwave_radiation':'shortwave_radiation_h','diffuse_radiation':'diffuse_radiation_h','surface_solar_radiation_downwards':'surface_solar_radiation_downwards_f'})
print(df_merging9.columns)

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id', 'dt_date',
       'dt_year', 'dt_quarter', 'dt_month', 'dt_week', 'dt_day_of_month',
       'dt_hour', 'day_of_year', 'day_of_week', 'eic_count',
       'installed_capacity', 'euros_per_mwh_electricity', 'euros_per_mwh_gas',
       'temperature_f', 'dewpoint_f', 'cloudcover_high_f', 'cloudcover_low_f',
       'cloudcover_mid_f', 'cloudcover_total_f', 'direct_solar_radiation_f',
       'surface_solar_radiation_downwards_f', 'snowfall_f', 'rain_f',
       'temperature_h', 'dewpoint_h', 'rain_h', 'snowfall_h',
       'cloudcover_total_h', 'cloudcover_low_h', 'cloudcover_mid_h',
       'cloudcover_high_h', 'shortwave_radiation_h',
       'direct_solar_radiation_h', 'diffuse_radiation_h'],
      dtype='object')


In [ ]:
## ele price outlier : 4000 -> 527.9
avg_ele_price=df_merging9[((df_merging7['dt_year']==2022)&(df_merging9['dt_month']==8)&(df_merging9['dt_hour']==17))]['euros_per_mwh_electricity'].mean()
avg_ele_price

In [60]:
## save file
df_merging9.to_csv('merged_dataset.csv')